# Running MegaDetector on camera trap images using Google Colab

[Open this notebook in Colab](https://colab.research.google.com/github/microsoft/CameraTraps/blob/main/detection/megadetector_colab.ipynb)

Adapted from previous versions by [@louis030195](https://github.com/louis030195) 
and [@alsnothome](https://github.com/alsnothome).

Also see the [MegaDetector guide on GitHub](https://github.com/ecologize/CameraTraps/blob/main/megadetector.md).

This notebook is designed to load camera trap images that have already been uploaded to Google Drive. If you don't have your own images on Google Drive, this notebook will show you how to download some sample images from [LILA](https://lila.science).

MegaDetector output is saved in a .json file whose format is described  [here](https://github.com/ecologize/CameraTraps/tree/main/api/batch_processing#batch-processing-api-output-format). The last cell in this notebook will give you some pointers on how users typically work with MegaDetector output.

## Set up the Colab instance to run on a GPU accelerator


Navigate to Edit→Notebook Settings and select "GPU" from the "Hardware accelerator" drop-down menu.

## Install dependencies, download the model, set up your PYTHONPATH

From here on, you'll start seeing a mix of code and Linux system commands. System commands are prefixed by a shebang `!`, which tells this notebook to execute them on the command line.

### Install required Python packages

This may take 2-3 minutes.

In [ ]:
pip install humanfriendly jsonpickle

In [ ]:
pip install torch==1.10.1 torchvision==0.11.2

### Download the MegaDetector model files

We'll download both MegaDetector v5a and v5b.  See the [release notes](https://github.com/ecologize/CameraTraps/releases/tag/v5.0) for information about the differences between the two models.

In [ ]:
!wget -O /content/md_v5a.0.0.pt https://github.com/ecologize/CameraTraps/releases/download/v5.0/md_v5a.0.0.pt
!wget -O /content/md_v5b.0.0.pt https://github.com/ecologize/CameraTraps/releases/download/v5.0/md_v5b.0.0.pt 

### Clone the required git repos
This will copy the latest version of the Microsoft AI for Earth "utilities" and "CameraTraps" repositories from GitHub, as well as the YOLOv5 repo, all of which are required to run MegaDetector.

In [ ]:
!git clone https://github.com/ecologize/CameraTraps
!git clone https://github.com/microsoft/ai4eutils
!git clone https://github.com/ultralytics/yolov5/
!cd yolov5 && git checkout c23a441c9df7ca9b1f275e8c8719c949269160d1

### Set `PYTHONPATH` to include `CameraTraps`, `ai4eutils`, and `yolov5`

Add cloned git folders to the `PYTHONPATH` environment variable so that we can import their modules from any working directory.


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/ai4eutils"
os.environ['PYTHONPATH'] += ":/content/CameraTraps"
os.environ['PYTHONPATH'] += ":/content/yolov5"

## Mount Google Drive in Colab
You can mount your Google Drive if you have your sample images there, or if want to save the results to your Google Drive.  

Once you run the cell below, you will be prompted to authorize Colab to access your Google Drive.  Your Google Drive folders will then be mounted under `/content/drive` and can be viewed and navigated in the Files pane in Colab.

The method is described in [this Colab code snippet](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download sample images

We install Microsoft's [azcopy](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10) utility, which we then use to download a few camera trap images from the [Snapshot Serengeti](http://lila.science/datasets/snapshot-serengeti) dataset hosted on [lila.science](http://lila.science).  If you are using your own data, you can skip this step.

In [ ]:
%%bash

# Download azcopy
wget -q -O azcopy_linux.tar.gz https://aka.ms/downloadazcopy-v10-linux
tar -xvzf azcopy_linux.tar.gz --wildcards */azcopy --strip 1
rm azcopy_linux.tar.gz
chmod u+x azcopy

# Copy a few Snapshot Serengeti images to a local directory
DATASET_URL="https://lilablobssc.blob.core.windows.net/snapshotserengeti-unzipped/"
SAMPLE_DIR="S1/D05/D05_R4"
LOCAL_DIR="/content/snapshotserengeti"

./azcopy cp "${DATASET_URL}${SAMPLE_DIR}" "${LOCAL_DIR}" --recursive

## MegaDetector batch processing

This step executes the Python script `run_detector_batch.py` from the CameraTraps repo. It has three mandatory arguments and one optional:

1. Path to the MegaDetector model file
2. A folder containing images.  This notebook points to the folder where we just put our Snapshot Serengeti images; if your images were already on Google Drive, replace `[Image_Folder]` with your folder name.
3. The output JSON file location and name.

In [ ]:
images_dir = '/content/snapshotserengeti'

# Choose a location for the output JSON file
output_file_path = '/content/drive/My Drive/snapshotserengeti-test/snapshot-serengeti-megadetector-results.json'

Here we pass the Python variable value `output_file_path` you specified above to the bash commands below using `$` (double quoting as there are spaces in this path), to run the script. This is so that we can refer to the output file path later for visualization.

# Run the detection script

There are actually two variants of MegaDetector v5, called "v5a" and "v5b".  By default this notebook runs MDv5a; change "md_v5a.0.0.pt" to "md_v5b.0.0.pt" to run MDv5b instead.

Both run at the same speed; if you are in a Colab session with a GPU accelerator, you should be able to process around four images per second.

In [ ]:
!python /content/CameraTraps/detection/run_detector_batch.py md_v5a.0.0.pt "$images_dir" "$output_file_path" --recursive --output_relative_filenames --quiet

## Visualize batch processing script outputs

Here we use the `visualize_detector_output.py` in the `visualization` folder of the Camera Traps repo to see the output of the MegaDetector visualized on our images. It will save images annotated with the results (original images will *not* be modified) to the folder you specify here.

The scripts take in a number of optional parameters to control output image size and how many are sampled (if you've processed a lot of images but only want to visualize the results on a few) - take a look at the `main()` function in the script to see what other parameters are available.

In [ ]:
# Render bounding boxes on our images
visualization_dir = '/content/visualized_images'
!python /content/CameraTraps/visualization/visualize_detector_output.py "$output_file_path" "$visualization_dir" --confidence 0.2 --images_dir "$images_dir"

In [ ]:
# Show the images with bounding boxes in Colab
import os
from PIL import Image

for viz_file_name in os.listdir(visualization_dir):
  print(viz_file_name)
  im = Image.open(os.path.join(visualization_dir, viz_file_name))
  display(im)

# Next steps

Now that you have run MegaDetector on a few images, here are some pointers to help you take advantage of MegaDetector to label your survey images more quickly.

### Ways to use the output .json in a camera trap image processing workflow 

#### 1. Timelapse

[Timelapse](http://saul.cpsc.ucalgary.ca/timelapse/pmwiki.php?n=Main.HomePage) is an open-source tool for annotating camera trap images. We have worked with the Timelapse developer to integrate MegaDetector results into Timelapse, so a user can:

- Select or sort images based on whether they contain animal or people or vehicles.
- View bounding boxes during additional manual annotation steps

See the [Timelapse Image Recognition Guide](https://saul.cpsc.ucalgary.ca/timelapse/uploads/Guides/TimelapseImageRecognitionGuide.pdf) for more information.

![Screenshot showing the Timelapse application with MegaDetector output, shown as a bounding box around the detected animal](https://github.com/ecologize/CameraTraps/blob/main/api/batch_processing/integration/images/tl_boxes.jpg?raw=1)


#### 2. Separating images into folders that contain animals/people/vehicles/nothing

Some MegaDetector users do image review without Timelapse, by moving the images to separate folders containing animals/people/vehicles/nothing according to MegaDetector output. You can use the script [separate_detections_into_folders.py](https://github.com/ecologize/CameraTraps/blob/master/api/batch_processing/postprocessing/separate_detections_into_folders.py) for this.

